In [42]:
import mysql.connector
import pandas as pd
import matplotlib as plt
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Set pandas display options to show more rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [22]:
# Connect to the MySQL database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456",
    database="cricket_data"
)

# Define the query to select all columns from the matches table
query = """SELECT matches.match_id, matches.gender, matches.start_date, matches.teams_type,
matches.match_type, matches.team_involved_one, matches.team_involved_two, info_section.outcome,
info_section.toss, info_section.venue
FROM matches JOIN
info_section ON
matches.match_id = info_section.match_id;"""

# Execute the query and fetch the results
cursor = connection.cursor()
cursor.execute(query)
matches_data = cursor.fetchall()

# Close the cursor and connection
cursor.close()
connection.close()

# Convert the fetched data to a pandas DataFrame
matches_df = pd.DataFrame(matches_data, columns=[desc[0] for desc in cursor.description])

In [23]:
matches_df.head()

,match_id,gender,start_date,teams_type,match_type,team_involved_one,team_involved_two,outcome,toss,venue
0,1000851,male,2016-11-03,international,Test,Australia,South Africa,"{""by"": {""runs"": 177}, ""winner"": ""South Africa""}","{""decision"": ""bat"", ""winner"": ""South Africa""}","""Western Australia Cricket Association Ground"""
1,1000853,male,2016-11-12,international,Test,Australia,South Africa,"{""by"": {""innings"": 1, ""runs"": 80}, ""winner"": ""...","{""decision"": ""field"", ""winner"": ""South Africa""}","""Bellerive Oval"""
2,1000855,male,2016-11-24,international,Test,Australia,South Africa,"{""by"": {""wickets"": 7}, ""winner"": ""Australia""}","{""decision"": ""bat"", ""winner"": ""South Africa""}","""Adelaide Oval"""
3,1000881,male,2016-12-15,international,Test,Australia,Pakistan,"{""by"": {""runs"": 39}, ""winner"": ""Australia""}","{""decision"": ""bat"", ""winner"": ""Australia""}","""Brisbane Cricket Ground, Woolloongabba"""
4,1000883,male,2016-12-26,international,Test,Australia,Pakistan,"{""by"": {""innings"": 1, ""runs"": 18}, ""winner"": ""...","{""decision"": ""bat"", ""winner"": ""Pakistan""}","""Melbourne Cricket Ground"""


In [24]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17188 entries, 0 to 17187
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   match_id           17188 non-null  object
 1   gender             17188 non-null  object
 2   start_date         17188 non-null  object
 3   teams_type         17188 non-null  object
 4   match_type         17188 non-null  object
 5   team_involved_one  17188 non-null  object
 6   team_involved_two  17188 non-null  object
 7   outcome            17188 non-null  object
 8   toss               17188 non-null  object
 9   venue              17188 non-null  object
dtypes: object(10)
memory usage: 1.3+ MB


In [25]:
matches_df.describe()

,match_id,gender,start_date,teams_type,match_type,team_involved_one,team_involved_two,outcome,toss,venue
count,17188,17188,17188,17188,17188,17188,17188,17188,17188,17188
unique,17188,2,5200,2,39,346,347,9207,690,797
top,1000851,male,2019-02-24,club,T20,England,Pakistan,"{""result"": ""draw""}","{""decision"": ""bat"", ""winner"": ""Australia""}","""Dubai International Cricket Stadium"""
freq,1,14255,25,9119,3560,634,635,774,349,322


In [26]:
def transform_dataframe(df):
    # 1. Change the start_date to datetime
    df['start_date'] = pd.to_datetime(df['start_date'])

    # 2. Split the toss field into toss_winner and toss_decision
    def extract_toss_details(toss):
        toss_data = json.loads(toss)  # Parse the JSON string to a dictionary
        return toss_data.get('winner'), toss_data.get('decision')
    
    df[['toss_winner', 'toss_decision']] = df['toss'].apply(lambda x: pd.Series(extract_toss_details(x)))

    # Drop the original toss column
    df.drop(columns=['toss'], inplace=True)

    # 3. Remove double quotes from the venue field
    df['venue'] = df['venue'].str.strip('"')

    return df



In [27]:
transformed_df = transform_dataframe(matches_df)

In [28]:
transformed_df

,match_id,gender,start_date,teams_type,match_type,team_involved_one,team_involved_two,outcome,venue,toss_winner,toss_decision
0,1000851,male,2016-11-03,international,Test,Australia,South Africa,"{""by"": {""runs"": 177}, ""winner"": ""South Africa""}",Western Australia Cricket Association Ground,South Africa,bat
1,1000853,male,2016-11-12,international,Test,Australia,South Africa,"{""by"": {""innings"": 1, ""runs"": 80}, ""winner"": ""...",Bellerive Oval,South Africa,field
2,1000855,male,2016-11-24,international,Test,Australia,South Africa,"{""by"": {""wickets"": 7}, ""winner"": ""Australia""}",Adelaide Oval,South Africa,bat
3,1000881,male,2016-12-15,international,Test,Australia,Pakistan,"{""by"": {""runs"": 39}, ""winner"": ""Australia""}","Brisbane Cricket Ground, Woolloongabba",Australia,bat
4,1000883,male,2016-12-26,international,Test,Australia,Pakistan,"{""by"": {""innings"": 1, ""runs"": 18}, ""winner"": ""...",Melbourne Cricket Ground,Pakistan,bat
...,...,...,...,...,...,...,...,...,...,...,...
17183,wi_211979,female,2022-06-21,club,SFT,Guyana,Leeward Islands,"{""winner"": ""Guyana"", ""by"": {""wickets"": 8}}","Guyana National Stadium, Providence",Leeward Islands,bat
17184,wi_212061,female,2022-06-22,club,SFT,Barbados,Trinidad and Tobago,"{""winner"": ""Barbados"", ""by"": {""wickets"": 9}, ""...","Guyana National Stadium, Providence",Trinidad and Tobago,bat
17185,wi_212062,female,2022-06-23,club,SFT,Guyana,Jamaica,"{""winner"": ""Jamaica"", ""by"": {""wickets"": 8}}","Guyana National Stadium, Providence",Jamaica,field
17186,wi_212063,female,2022-06-24,club,SFT,Windward Islands,Leeward Islands,"{""winner"": ""Windward Islands"", ""by"": {""runs"": ...","Guyana National Stadium, Providence",Windward Islands,bat


In [29]:
def process_outcome(outcome):
    # Initialize default values
    result = winner = runs = innings = wickets = method = None

    # Parse the JSON string to a dictionary
    outcome_data = json.loads(outcome)

    # Extract the common fields
    if 'result' in outcome_data:
        result = outcome_data['result']
    if 'winner' in outcome_data:
        winner = outcome_data['winner']
    if 'method' in outcome_data:
        method = outcome_data['method']
    if 'by' in outcome_data:
        by = outcome_data['by']
        if 'runs' in by:
            runs = by['runs']
        if 'innings' in by:
            innings = by['innings']
        if 'wickets' in by:
            wickets = by['wickets']

    return result, winner, runs, innings, wickets, method

# Apply the function to the outcome column and create new columns
matches_df[['result', 'winner', 'runs', 'innings', 'wickets', 'method']] = matches_df['outcome'].apply(
    lambda x: pd.Series(process_outcome(x))
)

# Drop the original outcome column
matches_df.drop(columns=['outcome'], inplace=True)

In [30]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17188 entries, 0 to 17187
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   match_id           17188 non-null  object        
 1   gender             17188 non-null  object        
 2   start_date         17188 non-null  datetime64[ns]
 3   teams_type         17188 non-null  object        
 4   match_type         17188 non-null  object        
 5   team_involved_one  17188 non-null  object        
 6   team_involved_two  17188 non-null  object        
 7   venue              17188 non-null  object        
 8   toss_winner        17188 non-null  object        
 9   toss_decision      17188 non-null  object        
 10  result             1301 non-null   object        
 11  winner             15887 non-null  object        
 12  runs               7969 non-null   float64       
 13  innings            367 non-null    float64       
 14  wicket

In [31]:
matches_df

,match_id,gender,start_date,teams_type,match_type,team_involved_one,team_involved_two,venue,toss_winner,toss_decision,result,winner,runs,innings,wickets,method
0,1000851,male,2016-11-03,international,Test,Australia,South Africa,Western Australia Cricket Association Ground,South Africa,bat,None,South Africa,177.0,NaN,NaN,None
1,1000853,male,2016-11-12,international,Test,Australia,South Africa,Bellerive Oval,South Africa,field,None,South Africa,80.0,1.0,NaN,None
2,1000855,male,2016-11-24,international,Test,Australia,South Africa,Adelaide Oval,South Africa,bat,None,Australia,NaN,NaN,7.0,None
3,1000881,male,2016-12-15,international,Test,Australia,Pakistan,"Brisbane Cricket Ground, Woolloongabba",Australia,bat,None,Australia,39.0,NaN,NaN,None
4,1000883,male,2016-12-26,international,Test,Australia,Pakistan,Melbourne Cricket Ground,Pakistan,bat,None,Australia,18.0,1.0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17183,wi_211979,female,2022-06-21,club,SFT,Guyana,Leeward Islands,"Guyana National Stadium, Providence",Leeward Islands,bat,None,Guyana,NaN,NaN,8.0,None
17184,wi_212061,female,2022-06-22,club,SFT,Barbados,Trinidad and Tobago,"Guyana National Stadium, Providence",Trinidad and Tobago,bat,None,Barbados,NaN,NaN,9.0,D/L
17185,wi_212062,female,2022-06-23,club,SFT,Guyana,Jamaica,"Guyana National Stadium, Providence",Jamaica,field,None,Jamaica,NaN,NaN,8.0,None
17186,wi_212063,female,2022-06-24,club,SFT,Windward Islands,Leeward Islands,"Guyana National Stadium, Providence",Windward Islands,bat,None,Windward Islands,151.0,NaN,NaN,None


In [44]:
# Group the DataFrame by the 'result' column and count the occurrences
method_group = matches_df.groupby('result').size()
method_group

result
draw         774
no result    370
tie          157
dtype: int64

In [32]:
# Group the DataFrame by the 'method' column and count the occurrences
method_group = matches_df.groupby('method').size()
method_group

method
Awarded                 3
D/L                   764
Lost fewer wickets      1
VJD                     5
dtype: int64

In [54]:
# Group the DataFrame by the 'venue' column and count the occurrences
method_group = matches_df.groupby('venue').size()
method_group

venue
7he Sevens Stadium                                                                  1
ACA Stadium                                                                        20
AMI Stadium                                                                        26
Abhimanyu Cricket Academy                                                          10
Abu Dhabi Oval 1                                                                    6
Achimota Senior Secondary School A Field                                           25
Adelaide Oval                                                                     182
Affies Park                                                                        19
Aigburth                                                                           13
Airforce Complex ground                                                            35
Al Amerat Cricket Ground Oman Cricket (Ministry Turf 1)                           161
Al Dhaid Cricket Village                        

In [48]:
# Group the DataFrame by the 'venue' column and count the occurrences
method_group = matches_df.groupby('venue').size()
method_group

venue
7he Sevens Stadium                                                                  1
ACA Stadium                                                                        20
AMI Stadium                                                                        26
Abhimanyu Cricket Academy                                                          10
Abu Dhabi Oval 1                                                                    6
Achimota Senior Secondary School A Field                                           25
Adelaide Oval                                                                     182
Affies Park                                                                        19
Aigburth                                                                           13
Airforce Complex ground                                                            35
Al Amerat Cricket Ground Oman Cricket (Ministry Turf 1)                           161
Al Dhaid Cricket Village                        

In [34]:
matches_df

,match_id,gender,start_date,teams_type,match_type,team_involved_one,team_involved_two,venue,toss_winner,toss_decision,result,winner,runs,innings,wickets,method
0,1000851,male,2016-11-03,international,Test,Australia,South Africa,Western Australia Cricket Association Ground,South Africa,bat,None,South Africa,177.0,NaN,NaN,None
1,1000853,male,2016-11-12,international,Test,Australia,South Africa,Bellerive Oval,South Africa,field,None,South Africa,80.0,1.0,NaN,None
2,1000855,male,2016-11-24,international,Test,Australia,South Africa,Adelaide Oval,South Africa,bat,None,Australia,NaN,NaN,7.0,None
3,1000881,male,2016-12-15,international,Test,Australia,Pakistan,"Brisbane Cricket Ground, Woolloongabba",Australia,bat,None,Australia,39.0,NaN,NaN,None
4,1000883,male,2016-12-26,international,Test,Australia,Pakistan,Melbourne Cricket Ground,Pakistan,bat,None,Australia,18.0,1.0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17183,wi_211979,female,2022-06-21,club,SFT,Guyana,Leeward Islands,"Guyana National Stadium, Providence",Leeward Islands,bat,None,Guyana,NaN,NaN,8.0,None
17184,wi_212061,female,2022-06-22,club,SFT,Barbados,Trinidad and Tobago,"Guyana National Stadium, Providence",Trinidad and Tobago,bat,None,Barbados,NaN,NaN,9.0,D/L
17185,wi_212062,female,2022-06-23,club,SFT,Guyana,Jamaica,"Guyana National Stadium, Providence",Jamaica,field,None,Jamaica,NaN,NaN,8.0,None
17186,wi_212063,female,2022-06-24,club,SFT,Windward Islands,Leeward Islands,"Guyana National Stadium, Providence",Windward Islands,bat,None,Windward Islands,151.0,NaN,NaN,None


In [35]:
match_type = matches_df.groupby('match_type').size()
match_type

match_type
APL       23
BBL      576
BLZ       17
BPL      389
BWT       47
CCH     1168
CEC      121
CPL      341
CTC      288
FRB       39
HND      187
ILT       66
IPL     1095
IPO       67
IPT       76
IT20     329
LPL       95
MDM       23
MLC       19
MSL       56
NTB     1254
ODI     2907
ODM      426
PKS      248
PSL      280
RHF      155
RLC      659
SAT       66
SFT        8
SMA      687
SSH      147
SSM      347
T20     3560
Test     824
WBB      435
WCL       11
WPL       44
WSL       95
WTC       13
dtype: int64

In [36]:
# Function to cluster similar venue names and choose a representative for each cluster
def standardize_venue_names(df, column_name, threshold=80):
    unique_venues = df[column_name].unique()
    clustered_venues = {}
    
    for venue in unique_venues:
        if venue not in clustered_venues:
            # Find all venues similar to the current venue
            similar_venues = process.extract(venue, unique_venues, scorer=fuzz.token_sort_ratio)
            similar_venues = [v for v in similar_venues if v[1] >= threshold]
            cluster = [v[0] for v in similar_venues]
            
            # Choose the shortest name in the cluster as the representative name
            representative = min(cluster, key=len)
            
            # Map all cluster members to the representative
            for v in cluster:
                clustered_venues[v] = representative
    
    # Replace all venue names in the DataFrame with their representative names
    df[column_name] = df[column_name].map(clustered_venues)
    
    return df

# Assume matches_df is already defined and contains the venue column
matches_df = standardize_venue_names(matches_df, 'venue')

# Display the standardized DataFrame
print(matches_df)

        match_id  gender start_date     teams_type match_type  \
0        1000851    male 2016-11-03  international       Test   
1        1000853    male 2016-11-12  international       Test   
2        1000855    male 2016-11-24  international       Test   
3        1000881    male 2016-12-15  international       Test   
4        1000883    male 2016-12-26  international       Test   
...          ...     ...        ...            ...        ...   
17183  wi_211979  female 2022-06-21           club        SFT   
17184  wi_212061  female 2022-06-22           club        SFT   
17185  wi_212062  female 2022-06-23           club        SFT   
17186  wi_212063  female 2022-06-24           club        SFT   
17187  wi_212064  female 2022-06-25           club        SFT   

      team_involved_one    team_involved_two  \
0             Australia         South Africa   
1             Australia         South Africa   
2             Australia         South Africa   
3             Australia    

In [37]:
# Group the DataFrame by the 'venue' column and count the occurrences
method_group = matches_df.groupby('venue').size()
method_group

venue
 Emerald Heights International School Ground        22
7he Sevens Stadium, Dubai                            1
ACA Stadium, Barsapara                              10
ACA Stadium,Mangalagiri                              7
AMI Stadium                                          9
                                                  ... 
Yeonhui Cricket Ground                              60
York Cricket Club                                    8
Zahur Ahmed Chowdhury Stadium                      164
Zayed Cricket Stadium                               63
Zhejiang University of Technology Cricket Field     15
Length: 565, dtype: int64

In [38]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Preprocess function to get the primary part of a venue name
def preprocess_venue_name(name):
    # Split by comma and take the first part
    return name.split(',')[0].strip()

# Function to cluster similar venue names and choose a representative for each cluster
def standardize_venue_names(df, column_name, threshold=80):
    unique_venues = df[column_name].unique()
    preprocessed_venues = {venue: preprocess_venue_name(venue) for venue in unique_venues}
    clustered_venues = {}
    
    for venue in unique_venues:
        if venue not in clustered_venues:
            primary_name = preprocessed_venues[venue]
            # Find all venues similar to the primary part of the current venue
            similar_venues = process.extract(primary_name, preprocessed_venues.values(), scorer=fuzz.token_sort_ratio)
            similar_venues = [v for v in similar_venues if v[1] >= threshold]
            cluster = [v[0] for v in similar_venues]
            
            # Choose the shortest name in the cluster as the representative name
            representative = min(cluster, key=len)
            
            # Map all cluster members to the representative
            for v in cluster:
                original_venues = [k for k, val in preprocessed_venues.items() if val == v]
                for ov in original_venues:
                    clustered_venues[ov] = representative
    
    # Replace all venue names in the DataFrame with their representative names
    df[column_name] = df[column_name].map(clustered_venues)
    
    return df

# Assume matches_df is already defined and contains the venue column
matches_df = standardize_venue_names(matches_df, 'venue')

# Display the standardized DataFrame
# print(matches_df)


In [43]:
# Group the DataFrame by the 'venue' column and count the occurrences
method_group = matches_df.groupby('venue').size()
method_group

venue
7he Sevens Stadium                                                                  1
ACA Stadium                                                                        20
AMI Stadium                                                                        26
Abhimanyu Cricket Academy                                                          10
Abu Dhabi Oval 1                                                                    6
Achimota Senior Secondary School A Field                                           25
Adelaide Oval                                                                     182
Affies Park                                                                        19
Aigburth                                                                           13
Airforce Complex ground                                                            35
Al Amerat Cricket Ground Oman Cricket (Ministry Turf 1)                           161
Al Dhaid Cricket Village                        